<a href="https://colab.research.google.com/github/MaxVortman/IDEA-code-clones/blob/master/colab/code_clones_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression

In [0]:
import pandas as pd
import numpy as np
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from scipy import sparse

import warnings
warnings.filterwarnings('ignore')

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

def printMetrics(y_test, y_pred):
    print('accuracy: ')
    print(accuracy_score(y_test, y_pred))
    print('\nprecision: ')
    print(precision_score(y_test, y_pred))
    print('\nrecall: ')
    print(recall_score(y_test, y_pred))
    print('\nroc auc: ')
    print(roc_auc_score(y_test, y_pred))

In [0]:
def resample(X, y):
  rus = RandomUnderSampler(random_state=23)
  return rus.fit_resample(X, y)

## Bag of words

In [31]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
sparse_bow_X = sparse.load_npz("/content/gdrive/My Drive/code-clones/csv/bagofwords_vectors_X.npz")
sparse_bow_y = sparse.load_npz("/content/gdrive/My Drive/code-clones/csv/bagofwords_vectors_y.npz").T

In [33]:
sparse_bow_y.shape

(19990, 1)

In [34]:
sparse_bow_X.shape

(19990, 52736)

In [0]:
X_resampled, y_resampled = resample(sparse_bow_X, sparse_bow_y.toarray())
#print(sorted(Counter(y_resampled.T).items()))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

In [36]:
!git clone https://github.com/hyperopt/hyperopt-sklearn
%cd hyperopt-sklearn
!pip install -e .

Cloning into 'hyperopt-sklearn'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1171 (delta 1), reused 4 (delta 0), pack-reused 1164
Receiving objects: 100% (1171/1171), 2.00 MiB | 21.13 MiB/s, done.
Resolving deltas: 100% (708/708), done.
/content/hyperopt-sklearn
Obtaining file:///content/hyperopt-sklearn
  Running setup.py develop for hpsklearn


In [37]:
from hpsklearn import HyperoptEstimator, any_sparse_classifier, any_classifier
from hyperopt import tpe

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


## Сustom loss

In [0]:
def calc_fnr_fpr(y_target, y_prediction):
  fp = 0
  fn = 0
  n = len(y_target)
  for i in range(n):
    if y_prediction[i] == 1 and y_target[i] == 0:
      fp += 1
    if y_prediction[i] == 0 and y_target[i] == 1:
      fn += 1
  return (fn / n, fp / n) 

In [0]:
def custom_loss(y_target, y_prediction):
  k_fn = 1
  k_fp = 10
  n = len(y_target)
  (fnr, fpr) = calc_fnr_fpr(y_target, y_prediction)
  return k_fn * fnr + k_fp * fpr

In [0]:
def hyperopt_custom_loss_estimator(X_train, y_train, sparse=True):
  classifier = any_sparse_classifier('clf') if sparse else any_classifier('clf')
  estim = HyperoptEstimator(classifier=classifier,
                          preprocessing=[],
                          algo=tpe.suggest, 
                          trial_timeout=300,
                          loss_fn=custom_loss,
                          seed=23)
  estim.fit(X_train, y_train)
  return estim

## Custom loss bow

In [41]:
estim = hyperopt_custom_loss_estimator(X_train, y_train)

100%|██████████| 1/1 [00:00<00:00, 15.16it/s, best loss: 1.4223449969306323]


In [0]:
y_pred = estim.predict(X_test)

In [43]:
calc_fnr_fpr(y_test, y_pred)

(0.1285451197053407, 0.1406998158379374)

In [46]:
printMetrics(y_test, y_pred)

accuracy: 
0.7307550644567219

precision: 
0.730225988700565

recall: 
0.74765003615329

roc auc: 
0.7304316246832515


In [47]:
estim.best_model()

{'ex_preprocs': (),
 'learner': SGDClassifier(alpha=0.06118428364662971, average=False, class_weight=None,
        early_stopping=False, epsilon=0.1, eta0=0.0003970052556398526,
        fit_intercept=True, l1_ratio=0.5022724095962902,
        learning_rate='constant', loss='log', max_iter=158684350.0,
        n_iter=None, n_iter_no_change=5, n_jobs=1, penalty='l2',
        power_t=0.907051629874976, random_state=0, shuffle=True,
        tol=0.006308140398304822, validation_fraction=0.1, verbose=False,
        warm_start=False),
 'preprocs': ()}

## code2vec

In [0]:
df1 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/lwjgl3_code2vec_vectors1.csv")
df2 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/lwjgl3_code2vec_vectors2.csv")
df3 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/lwjgl_code2vec_vectors.csv")
df4 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/spring_code2vec_vectors.csv")
df5 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/jenkins_code2vec_vectors.csv")
df6 = pd.read_csv("/content/gdrive/My Drive/code-clones/csv/rxjava_code2vec_vectors.csv")
df_c2v = pd.concat([df1, df2, df3, df4, df5, df6], axis=0)

In [49]:
df_c2v.head()

,exp,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,384
0,0.0,0.708866,-0.139283,0.790398,0.880303,0.291317,0.553174,0.823598,-0.693108,-0.379951,...,0.409421,-0.743124,-0.406415,0.348190,-0.722999,0.883331,0.975385,0.811376,0.529098,-0.580801
1,0.0,0.457570,0.465724,0.049870,0.329546,0.185015,-0.367061,0.646942,0.066054,-0.377213,...,0.422311,0.542749,0.726254,0.081205,-0.139942,-0.282454,0.584696,0.316820,0.038821,-0.080841
2,0.0,0.674882,-0.227605,-0.051460,0.322587,0.113033,-0.206298,0.618875,-0.831079,0.467831,...,0.380035,-0.358241,0.168523,-0.115633,0.013167,-0.600655,-0.152489,0.309746,0.104927,-0.338480
3,0.0,0.238518,0.239017,-0.329432,0.076081,0.674525,0.240891,-0.138851,0.377708,-0.052835,...,-0.629672,-0.818671,-0.240460,0.834071,-0.167629,-0.331171,-0.589584,0.668989,0.260057,0.465806
4,0.0,-0.261068,-0.683279,-0.386828,0.031994,-0.891767,0.202983,-0.022529,0.667178,0.515003,...,-0.393192,0.125753,0.020193,0.403067,-0.779176,0.066885,0.953279,-0.108147,0.251376,-0.543750


In [0]:
y = df_c2v['exp']
X = df_c2v.drop('exp', 1)
X_resampled, y_resampled = resample(X, y)
#print(sorted(Counter(y_resampled.T).items()))
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=23)

## Custom loss code2vec

In [51]:
estim = hyperopt_custom_loss_estimator(X_train, y_train, sparse=False)

100%|██████████| 1/1 [00:03<00:00,  3.91s/it, best loss: 1.8229768786127167]


In [0]:
y_pred = estim.predict(X_test)

In [53]:
calc_fnr_fpr(y_test, y_pred)

(0.07368877329865627, 0.19592544429995665)

In [54]:
printMetrics(y_test, y_pred)

accuracy: 
0.7303857824013871

precision: 
0.6861111111111111

recall: 
0.853195164075993

roc auc: 
0.7299048057107556


In [55]:
estim.best_model()

{'ex_preprocs': (),
 'learner': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.005891891362147046, loss='deviance',
               max_depth=None, max_features='log2', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=8, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=16,
               n_iter_no_change=None, presort='auto', random_state=0,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 'preprocs': ()}